<a href="https://colab.research.google.com/github/joyc3lin/sqlite_database_operations/blob/main/HHA504_week3assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#loading in packages
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

# **1. Data Exploration and Analysis**

In [2]:
# importing pricing transparency datasets
df_msmc = pd.read_excel('datasets/MountSinaiMedicalCenter_standardcharges.xlsx')
df_nyp = pd.read_json('datasets/NewYorkPresbyterianHospital_standardcharges.json')

In [3]:
print(df_msmc.shape)
df_msmc

(3577, 56)


,Charge Description,Long Description,Billing Code,Revenue Code / DRG Type,Gross Charge,Cash Price,AETNA BETTER HEALTH (MEDICAID),AETNA COMMERCIAL,AETNA INTERNATIONAL,AETNA MEDICARE,...,UNITED HEALTHCARE,UNITED HEALTHCARE MEDICARE / MEDICA / PREFERRED CARE PARTNERS,UNITED HEALTHCARE OPTIONS PPO,UNITED HEALTHCARE MEDICAID,WELLCARE MEDICAID,WELLCARE MEDICARE,WORKERS COMP,WW CONCIERGE,Maximum Negotiated Charge,Minimum Negotiated Charge
0,HC BRIDGE ASSESSMENT,Psychiatric diagnostic evaluation\n,90791,513,156.000000,175,10.920,46.800000,132.600000,21.840000,...,32.760000,20.280000,34.320000,10.920,10.920,17.160000,117.000000,78.000000,132.600000,10.920000
1,HC PSYCHIATRIC DIAGNOSTIC EVALUATION,Psychiatric diagnostic evaluation\n,90791,900,327.600000,315,22.932,98.280000,278.460000,45.864000,...,68.796000,42.588000,72.072000,22.932,22.932,36.036000,245.700000,163.800000,278.460000,22.932000
2,HC PSYCHIATRIC DIAGNOSTIC EVALUATION W/ MEDICA...,Psychiatric diagnostic evaluation with medical...,90792,900,327.600000,315,22.932,98.280000,278.460000,45.864000,...,68.796000,42.588000,72.072000,22.932,22.932,36.036000,245.700000,163.800000,278.460000,22.932000
3,HC ELECTROCONVULSIVE THERAPY,Electroconvulsive therapy (includes necessary ...,90870,901,1154.400000,577.2,80.808,346.320000,981.240000,161.616000,...,242.424000,150.072000,253.968000,80.808,80.808,126.984000,865.800000,577.200000,981.240000,80.808000
4,HC PSYCH TREATMENT & EDUCATION,Training and educational services related to t...,G0177,942,166.400000,83.2,11.648,49.920000,141.440000,23.296000,...,34.944000,21.632000,36.608000,11.648,11.648,18.304000,124.800000,83.200000,141.440000,11.648000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3572,Extensive O.R. procedure unrelated to principa...,Extensive O.R. procedure unrelated to principa...,981,MS,176654.394366,110622.909091,NaN,52996.318310,150156.235211,24731.615211,...,37097.422817,22965.071268,38863.966761,NaN,NaN,19431.983380,132490.795775,88327.197183,150156.235211,14132.351549
3573,Extensive O.R. procedure unrelated to principa...,Extensive O.R. procedure unrelated to principa...,982,MS,105720.700000,33313.777778,NaN,31716.210000,89862.595000,14800.898000,...,22201.347000,13743.691000,23258.554000,NaN,NaN,11629.277000,79290.525000,52860.350000,89862.595000,8457.656000
3574,Extensive O.R. procedure unrelated to principa...,Extensive O.R. procedure unrelated to principa...,983,MS,45953.375000,23374.75,NaN,13786.012500,39060.368750,6433.472500,...,9650.208750,5973.938750,10109.742500,NaN,NaN,5054.871250,34465.031250,22976.687500,39060.368750,3676.270000
3575,Non-extensive O.R. proc unrelated to principal...,Non-extensive O.R. proc unrelated to principal...,987,MS,196679.153846,44923.2,NaN,59003.746154,167177.280769,27535.081538,...,41302.622308,25568.290000,43269.413846,NaN,NaN,21634.706923,147509.365385,98339.576923,167177.280769,15734.332308


In [4]:
print(df_nyp.shape)
df_nyp

(6193, 47)


,Code (CPT/DRG),Description,Rev Code,Inpatient/Outpatient,Gross Charges,Discounted Cash Price,Aetna,Cigna,Empire Blue Cross Blue Shield,Emblem Health,...,Consumer Health Network,Devon,Equian,First Health,Magnacare,Multiplan/Beechstreet/PHCS,QHM,Worldwide,Minimum Negotiated Charge,Maximum Negotiated Charge
0,96360,HC IV INFUSION HYDRATION INITIAL 31 MIN-1HR,0260,Inpatient/Outpatient,866.000000,866.000000,641.706,494.486,640.84,1388.424,...,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,253.048250,1388.424000
1,96361,HC IV INFUSION HYDRATION FLUIDS ADDL HR,0260,Inpatient/Outpatient,358.000000,358.000000,265.278,204.418,264.92,1305.112,...,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,51.903250,1990.891341
2,96365,HC IV INFUSION FOR THER/PROPH/DIAG INITIAL UP ...,0260,Inpatient/Outpatient,1397.000000,1397.000000,1035.177,797.687,1033.78,1475.508,...,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,253.048250,1475.508000
3,96367,HC IV INF THER/PROPH/DIAG ADDL SEQ NEW DRUG UP...,0260,Inpatient/Outpatient,480.000000,480.000000,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,82.650750,1325.120000
4,96368,HC IV INFUSION FOR THER/PROPH/DIAG CONCURRENT,0260,Inpatient/Outpatient,480.000000,480.000000,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,156.982800,1325.120000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Multiple,Inpatient,184095.129020,184095.129020,86723.5232,98911.694706,82299.84,115129.89148,...,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,12561.144027,115129.891480
6189,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Multiple,Inpatient,85385.941857,85385.941857,57748.8352,65864.888166,63931.455093,76664.51828,...,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,7833.746854,76664.518280
6190,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Multiple,Inpatient,411665.999955,411665.999955,114736.9984,130862.199072,214294.876425,152319.20576,...,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,21016.798830,214294.876425
6191,988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Multiple,Inpatient,126522.871140,126522.871140,58654.7264,66898.093812,91907.6928,77867.13496,...,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,10429.428059,91907.692800


## Data Cleaning

### *Cleaning df_nyp*

In [5]:
# checking data types
for c in df_nyp.columns:
  print(f"{c}: {type(df_nyp[c][0])}")

Code (CPT/DRG): <class 'int'>
Description: <class 'str'>
Rev Code: <class 'str'>
Inpatient/Outpatient: <class 'str'>
Gross Charges: <class 'numpy.float64'>
Discounted Cash Price: <class 'numpy.float64'>
Aetna: <class 'float'>
Cigna: <class 'float'>
Empire Blue Cross Blue Shield: <class 'float'>
Emblem Health: <class 'float'>
United Health Group: <class 'float'>
Aetna Medicare: <class 'float'>
AgeWell Medicare: <class 'float'>
Emblem Medicare: <class 'float'>
Empire Medicare: <class 'float'>
Fidelis Medicare: <class 'float'>
Healthfirst Medicare: <class 'float'>
UHC Community Plan/United Medicare: <class 'float'>
VNS Medicare: <class 'float'>
WellCare Medicare: <class 'float'>
1199: <class 'int'>
Affinity Molina Essential: <class 'float'>
Affinity Molina Medicaid/CHP: <class 'float'>
Amida Care Medicaid: <class 'float'>
Emblem Medicaid/CHP: <class 'float'>
Empire Healthplus Essential: <class 'float'>
Empire Healthplus Exchange: <class 'float'>
Empire Healthplus Medicaid/CHP: <class 'flo

In [6]:
# looking for missing values
missing_values = df_nyp.isnull().sum()
missing_values

Code (CPT/DRG)                          0
Description                             0
Rev Code                                0
Inpatient/Outpatient                    0
Gross Charges                           0
Discounted Cash Price                   0
Aetna                                   0
Cigna                                   0
Empire Blue Cross Blue Shield           0
Emblem Health                           0
United Health Group                     0
Aetna Medicare                          0
AgeWell Medicare                        0
Emblem Medicare                         0
Empire Medicare                         0
Fidelis Medicare                        0
Healthfirst Medicare                    0
UHC Community Plan/United Medicare      0
VNS Medicare                            0
WellCare Medicare                       0
1199                                    0
Affinity Molina Essential               0
Affinity Molina Medicaid/CHP            0
Amida Care Medicaid               

In [7]:
# dropping row with missing data
df_nyp.dropna(inplace=True)
df_nyp.shape

(5912, 47)

In [8]:
#removing duplicate rows and columns
df_nyp.drop_duplicates(inplace=True)
df_nyp.loc[:,~df_nyp.T.duplicated(keep='first')]
print(df_nyp.shape)

(5912, 47)


In [9]:
# cleaning column names

# Function to remove white space and special characters from a value
def clean_value(value):
    cleaned_value = re.sub(r'\s+|[^a-zA-Z0-9]', '', str(value))
    return cleaned_value

df_nyp.rename(mapper=clean_value, axis=1, inplace=True)
df_nyp.sample(10)

,CodeCPTDRG,Description,RevCode,InpatientOutpatient,GrossCharges,DiscountedCashPrice,Aetna,Cigna,EmpireBlueCrossBlueShield,EmblemHealth,...,ConsumerHealthNetwork,Devon,Equian,FirstHealth,Magnacare,MultiplanBeechstreetPHCS,QHM,Worldwide,MinimumNegotiatedCharge,MaximumNegotiatedCharge
3673,11300,HC SHAVE LESION SKIN TRUNK/ARM/LEG UP TO 0.5CM,0510,Inpatient/Outpatient,503.0,503.0,Not separately payable,Not separately payable,352.1,Not separately payable,...,228.5632,228.5632,228.5632,228.5632,228.5632,228.5632,228.5632,228.5632,228.563200,995.326806
147,82157,HC LAB ASSAY OF ANDROSTENEDIONE,0301,Inpatient/Outpatient,455.0,455.0,20.496,24.773,20.496,20.496,...,206.752,206.752,206.752,206.752,206.752,206.752,206.752,206.752,20.496000,273.000000
4706,D7920,HC DENTAL SKIN GRAFT PROCEDURE,0512,Inpatient/Outpatient,1514.0,1514.0,Not separately payable,Not separately payable,1059.8,Not separately payable,...,687.9616,687.9616,687.9616,687.9616,687.9616,687.9616,687.9616,687.9616,457.183457,1211.200000
4118,D0371,HC DENTAL SIALOENDOSCOPY W/INTERPRETATION,0512,Inpatient/Outpatient,355.0,355.0,Not separately payable,Not separately payable,248.5,Not separately payable,...,161.312,161.312,161.312,161.312,161.312,161.312,161.312,161.312,158.377345,395.943362
4726,D7963,HC DENTAL FRENULOPLASTY,0512,Inpatient/Outpatient,1080.0,1080.0,Not separately payable,Not separately payable,756,Not separately payable,...,490.752,490.752,490.752,490.752,490.752,490.752,490.752,490.752,211.395459,864.000000
1815,26040,RELEASE PALM CONTRACTURE,0360,Inpatient/Outpatient,21035.0,21035.0,9144,7496,11924,9474.4,...,9558.304,9558.304,9558.304,9558.304,9558.304,9558.304,9558.304,9558.304,1479.921898,12621.000000
1297,78645,HC NM CSF IMAGING SHUNT EVALUATION,0341,Inpatient/Outpatient,2710.0,2710.0,2008.11,1149.04,3187.492,1675.5209,...,1231.424,1231.424,1231.424,1231.424,1231.424,1231.424,1231.424,1231.424,384.370000,3187.492000
1738,23931,HC I&D DEEP BURSA UPPER ARM OR ELBOW,0360,Inpatient/Outpatient,24890.0,24890.0,9144,7496,11924,9474.4,...,11310.016,11310.016,11310.016,11310.016,11310.016,11310.016,11310.016,11310.016,1273.026920,14934.000000
1656,21299,CRANIO/MAXILLOFACIAL SURGERY,0360,Inpatient/Outpatient,35268.0,35268.0,9144,7496,Not separately payable,9474.4,...,16025.7792,16025.7792,16025.7792,16025.7792,16025.7792,16025.7792,16025.7792,16025.7792,254.591750,21160.800000
611,85525,HC LAB HEPARIN NEUTRALIZATION,0305,Inpatient/Outpatient,134.0,134.0,8.288,9.905,8.288,8.288,...,60.8896,60.8896,60.8896,60.8896,60.8896,60.8896,60.8896,60.8896,8.288000,219.842174


### Cleaing df_msmc

In [10]:
# checking data types
for c in df_msmc.columns:
  print(f"{c}: {type(df_msmc[c][0])}")

Charge Description: <class 'str'>
Long Description: <class 'str'>
Billing Code: <class 'int'>
Revenue Code / DRG Type: <class 'str'>
Gross Charge: <class 'numpy.float64'>
Cash Price: <class 'int'>
AETNA BETTER HEALTH (MEDICAID): <class 'numpy.float64'>
AETNA COMMERCIAL: <class 'numpy.float64'>
AETNA INTERNATIONAL: <class 'numpy.float64'>
AETNA MEDICARE: <class 'numpy.float64'>
AMBETTER: <class 'numpy.float64'>
AMERIGROUP: <class 'numpy.float64'>
BEECH STREET: <class 'numpy.float64'>
BCBS HEALTH OPTIONS HMO: <class 'numpy.float64'>
BCBS MEDICARE PPO: <class 'numpy.float64'>
BCBS HEALTH OPTIONS MCR: <class 'numpy.float64'>
BCBS NETWORK BLUE: <class 'numpy.float64'>
BCBS PPO: <class 'numpy.float64'>
BCBS PHS (TRADITIONAL): <class 'numpy.float64'>
BRIGHT HEALTH: <class 'numpy.float64'>
BRIGHT HEALTH MEDICARE: <class 'numpy.float64'>
CAREPLUS MEDICARE: <class 'numpy.float64'>
CIGNA : <class 'numpy.float64'>
CORVEL: <class 'numpy.float64'>
COVENTRY (FIRST HEALTH): <class 'numpy.float64'>
COV

In [11]:
# looking for missing values
missing_values2 = df_msmc.isnull().sum()
print(missing_values2)

Charge Description                                                 0
Long Description                                                   0
Billing Code                                                     729
Revenue Code / DRG Type                                            0
Gross Charge                                                       0
Cash Price                                                         0
AETNA BETTER HEALTH (MEDICAID)                                   572
AETNA COMMERCIAL                                                 270
AETNA INTERNATIONAL                                              270
AETNA MEDICARE                                                   270
AMBETTER                                                         270
AMERIGROUP                                                         0
BEECH STREET                                                     270
BCBS HEALTH OPTIONS HMO                                          270
BCBS MEDICARE PPO                 

In [12]:
# dropping row with missing data
df_msmc.dropna(inplace=True)
df_msmc.shape

(2006, 56)

In [13]:
#removing duplicate rows and columns
df_msmc.drop_duplicates(inplace=True)
df_msmc.loc[:,~df_msmc.T.duplicated(keep='first')]
print(df_msmc.shape)

(2003, 56)


In [14]:
# cleaning column names
df_msmc.rename(mapper=clean_value, axis=1, inplace=True)
df_msmc.sample(10)

,ChargeDescription,LongDescription,BillingCode,RevenueCodeDRGType,GrossCharge,CashPrice,AETNABETTERHEALTHMEDICAID,AETNACOMMERCIAL,AETNAINTERNATIONAL,AETNAMEDICARE,...,UNITEDHEALTHCARE,UNITEDHEALTHCAREMEDICAREMEDICAPREFERREDCAREPARTNERS,UNITEDHEALTHCAREOPTIONSPPO,UNITEDHEALTHCAREMEDICAID,WELLCAREMEDICAID,WELLCAREMEDICARE,WORKERSCOMP,WWCONCIERGE,MaximumNegotiatedCharge,MinimumNegotiatedCharge
1303,HC XR TIBIA/FIBULA AP&LAT,"Radiologic examination; tibia and fibula, 2 views",73590,320,776.78,125,54.3746,233.034,660.2630,108.7492,...,163.1238,100.9814,170.8916,54.3746,54.3746,85.4458,582.5850,388.390,660.2630,54.3746
1200,HC SURG PATH LEVEL II,"Level II - Surgical pathology, gross and micro...",88302,312,312.03,156.015,21.8421,93.609,265.2255,43.6842,...,65.5263,40.5639,68.6466,21.8421,21.8421,34.3233,234.0225,156.015,265.2255,21.8421
1233,HC XRA FEMUR AP & LAT LEFT,"Radiologic examination, femur; minimum 2 views",73552,320,1066.50,125,74.6550,319.950,906.5250,149.3100,...,223.9650,138.6450,234.6300,74.6550,74.6550,117.3150,799.8750,533.250,906.5250,74.6550
2466,HC LOW SKINSUB TR/AR/LE<100SQ CM,Application of low cost skin substitute graft ...,C5271,510,1170.90,585.45,81.9630,351.270,995.2650,163.9260,...,245.8890,152.2170,257.5980,81.9630,81.9630,128.7990,878.1750,585.450,995.2650,81.9630
825,HC LYME DISEASE WB,Antibody; Borrelia burgdorferi (Lyme disease) ...,86617,302,39.52,38,2.7664,11.856,33.5920,5.5328,...,8.2992,5.1376,8.6944,2.7664,2.7664,4.3472,29.6400,19.760,33.5920,2.7664
2413,HC INTRA-ATRIAL PACING,Intra-atrial pacing,93610,481,13260.00,6630,928.2000,3978.000,11271.0000,1856.4000,...,2784.6000,1723.8000,2917.2000,928.2000,928.2000,1458.6000,9945.0000,6630.000,11271.0000,928.2000
2713,HC US DUP UPPER EXT VEINS LTD,Duplex scan of extremity veins including respo...,93971,921,1526.17,763.085,106.8319,457.851,1297.2445,213.6638,...,320.4957,198.4021,335.7574,106.8319,106.8319,167.8787,1144.6275,763.085,1297.2445,106.8319
196,"HC TOBRAMYCIN, TROUGH",Tobramycin,80200,300,24.47,23.53,1.7129,7.341,20.7995,3.4258,...,5.1387,3.1811,5.3834,1.7129,1.7129,2.6917,18.3525,12.235,20.7995,1.7129
2309,HC ST TREAT OF SWALLOW DLY,Treatment of swallowing dysfunction and/or ora...,92526,440,399.55,199.775,27.9685,119.865,339.6175,55.9370,...,83.9055,51.9415,87.9010,27.9685,27.9685,43.9505,299.6625,199.775,339.6175,27.9685
1328,HC CHOLANGIOGRAM INTRAOP,Cholangiography and/or pancreatography; intrao...,74300,320,1528.10,764.05,106.9670,458.430,1298.8850,213.9340,...,320.9010,198.6530,336.1820,106.9670,106.9670,168.0910,1146.0750,764.050,1298.8850,106.9670


## data distribution and statistics

### *df_nyp*

In [15]:
# distribution by RevCode
df_nyp.groupby(['RevCode']).describe()

GrossCharges                                              \
                count           mean            std           min   
RevCode                                                             
0260             12.0     753.500000     404.300406    357.000000   
0270             10.0      67.200000      61.372813      2.000000   
0274              2.0    1003.500000     341.532575    762.000000   
0278             10.0    4298.600000   12854.565321     83.000000   
0300             26.0    3534.076923    7531.502899     36.000000   
...               ...            ...            ...           ...   
0942              8.0     170.250000     138.450352     65.000000   
0943              2.0     388.000000       9.899495    381.000000   
0944              1.0     131.000000            NaN    131.000000   
0948              2.0     197.000000       0.000000    197.000000   
Multiple        723.0  180613.709921  223844.512646  16861.788766   

                                                                   \
                  25%            50%            75%           max   
RevCode                                                             
0260        480.00000     556.000000     991.000000  1.397000e+03   
0270         26.25000      56.500000      85.000000  1.890000e+02   
0274        882.75000    1003.500000    1124.250000  1.245000e+03   
0278        116.50000     166.500000     364.250000  4.087900e+04   
0300        112.50000     535.000000    2752.750000  2.815100e+04   
...               ...            ...            ...           ...   
0942         87.75000      90.000000     219.250000  4.550000e+02   
0943        384.50000     388.000000     391.500000  3.950000e+02   
0944        131.00000     131.000000     131.000000  1.310000e+02   
0948        197.00000     197.000000     197.000000  1.970000e+02   
Multiple  73238.72433  114634.634066  194664.056085  2.538980e+06   

         DiscountedCashPrice                 ... MinimumNegotiatedCharge  \
                       count           mean  ...                     75%   
RevCode                                      ...                           
0260                    12.0     753.500000  ...              253.048250   
0270                    10.0      67.200000  ...               38.624000   
0274                     2.0    1003.500000  ...              337.500000   
0278                    10.0    4298.600000  ...              131.130000   
0300                    26.0    3534.076923  ...              332.605219   
...                      ...            ...  ...                     ...   
0942                     8.0     170.250000  ...               41.885000   
0943                     2.0     388.000000  ...               94.574850   
0944                     1.0     131.000000  ...               59.526400   
0948                     2.0     197.000000  ...               70.413000   
Multiple               723.0  180613.709921  ...            14066.153064   

                      MaximumNegotiatedCharge                                \
                  max                   count           mean            std   
RevCode                                                                       
0260         551.8625                    12.0    1482.603223     245.133340   
0270          85.8816                    10.0      54.028800      49.343742   
0274         358.5600                     2.0     455.990400     155.192402   
0278       11773.1520                    10.0    6212.815508   10714.042758   
0300         532.0000                    26.0    2793.977263    6051.885289   
...               ...                     ...            ...            ...   
0942         206.7520                     8.0     314.472284     120.306293   
0943         103.7530                     2.0     311.952000       7.959194   
0944          59.5264                     1.0     468.948267            NaN   
0948          70.4130                     2.0     182.981700  

In [16]:
# value_counts of description
desc_counts = df_nyp['Description'].value_counts()
print(desc_counts)


KNEE ARTHROSCOPY/SURGERY                                                         16
CYSTOSCOPY AND TREATMENT                                                         12
REPAIR EYELID DEFECT                                                             12
TREAT HUMERUS FRACTURE                                                            9
SHOULDER ARTHROSCOPY/SURGERY                                                      7
                                                                                 ..
PARTIAL REMOVAL LEG BONE(S)                                                       1
REMOVE FEMUR LESION/GRAFT                                                         1
REMOVE FEMUR LESION                                                               1
EXC THIGH/KNEE TUM DEP 5CM/>                                                      1
NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIPAL DIAGNOSIS WITHOUT CC/MCC     1
Name: Description, Length: 5652, dtype: int64


In [17]:
# value_counts of InpatientOutpatient
io_counts = df_nyp['InpatientOutpatient'].value_counts()
print(io_counts)

Inpatient/Outpatient    5189
Inpatient                723
Name: InpatientOutpatient, dtype: int64


In [18]:
# value_counts of CodeCPTDRG
code_counts = df_nyp['CodeCPTDRG'].value_counts()
print(code_counts)

96360    1
54056    1
57105    1
57065    1
56606    1
        ..
27827    1
27823    1
27822    1
27814    1
989      1
Name: CodeCPTDRG, Length: 5912, dtype: int64


In [19]:
# statistics of GrossCharges
print(f"The mean of GrossCharges is : {df_nyp['GrossCharges'].mean()}")
print(f"The median of GrossCharges is : {df_nyp['GrossCharges'].median()}")
print(f"The mode of GrossCharges is : {df_nyp['GrossCharges'].mode().iloc[0]}")
print(f"The range of GrossCharges is: {df_nyp['GrossCharges'].max() - df_nyp['GrossCharges'].min()}")
print(f"The variance of GrossCharges is: {df_nyp['GrossCharges'].var()}")
print(f"The standard deviation of GrossCharges is: {df_nyp['GrossCharges'].std()}")
print(f"The IQR of GrossCharges is: {df_nyp['GrossCharges'].quantile(0.75) - df_nyp['GrossCharges'].quantile(0.25)}")

The mean of GrossCharges is : 33550.80400420582
The median of GrossCharges is : 3968.0
The mode of GrossCharges is : 122.0
The range of GrossCharges is: 2538978.617631959
The variance of GrossCharges is: 9445426925.583761
The standard deviation of GrossCharges is: 97187.58627306145
The IQR of GrossCharges is: 31052.5


In [20]:
# statistics of DiscountedCashPrice
print(f"The mean of DiscountedCashPrice is : {df_nyp['DiscountedCashPrice'].mean()}")
print(f"The median of DiscountedCashPrice is : {df_nyp['DiscountedCashPrice'].median()}")
print(f"The mode of DiscountedCashPrice is : {df_nyp['DiscountedCashPrice'].mode().iloc[0]}")
print(f"The range of DiscountedCashPrice is: {df_nyp['DiscountedCashPrice'].max() - df_nyp['DiscountedCashPrice'].min()}")
print(f"The variance of DiscountedCashPrice is: {df_nyp['DiscountedCashPrice'].var()}")
print(f"The standard deviation of DiscountedCashPrice is: {df_nyp['DiscountedCashPrice'].std()}")
print(f"The IQR of DiscountedCashPrice is: {df_nyp['DiscountedCashPrice'].quantile(0.75) - df_nyp['DiscountedCashPrice'].quantile(0.25)}")

The mean of DiscountedCashPrice is : 33550.80400420582
The median of DiscountedCashPrice is : 3968.0
The mode of DiscountedCashPrice is : 122.0
The range of DiscountedCashPrice is: 2538978.617631959
The variance of DiscountedCashPrice is: 9445426925.583761
The standard deviation of DiscountedCashPrice is: 97187.58627306145
The IQR of DiscountedCashPrice is: 31052.5


In [21]:
# statistics of MinimumNegotiatedCharge
print(f"The mean of MinimumNegotiatedCharge	is : {df_nyp['MinimumNegotiatedCharge'].mean()}")
print(f"The median of MinimumNegotiatedCharge	is : {df_nyp['MinimumNegotiatedCharge'].median()}")
print(f"The mode of MinimumNegotiatedCharge	is : {df_nyp['MinimumNegotiatedCharge'].mode().iloc[0]}")
print(f"The range of MinimumNegotiatedCharge is: {df_nyp['MinimumNegotiatedCharge'].max() - df_nyp['MinimumNegotiatedCharge'].min()}")
print(f"The variance of MinimumNegotiatedCharge is: {df_nyp['MinimumNegotiatedCharge'].var()}")
print(f"The standard deviation of MinimumNegotiatedCharge is: {df_nyp['MinimumNegotiatedCharge'].std()}")
print(f"The IQR of MinimumNegotiatedCharge is: {df_nyp['MinimumNegotiatedCharge'].quantile(0.75) - df_nyp['MinimumNegotiatedCharge'].quantile(0.25)}")

The mean of MinimumNegotiatedCharge	is : 2249.9188801499627
The median of MinimumNegotiatedCharge	is : 431.1429045953
The mode of MinimumNegotiatedCharge	is : 2412.0
The range of MinimumNegotiatedCharge is: 204860.6716
The variance of MinimumNegotiatedCharge is: 36188853.47582183
The standard deviation of MinimumNegotiatedCharge is: 6015.717203777272
The IQR of MinimumNegotiatedCharge is: 1984.7715625


In [22]:
# statistics of MaximumNegotiatedCharge
print(f"The mean of MaximumNegotiatedCharge	is : {df_nyp['MaximumNegotiatedCharge'].mean()}")
print(f"The median of MaximumNegotiatedCharge	is : {df_nyp['MaximumNegotiatedCharge'].median()}")
print(f"The mode of MaximumNegotiatedCharge	is : {df_nyp['MaximumNegotiatedCharge'].mode().iloc[0]}")
print(f"The range of MaximumNegotiatedCharge is: {df_nyp['MaximumNegotiatedCharge'].max() - df_nyp['MaximumNegotiatedCharge'].min()}")
print(f"The variance of MaximumNegotiatedCharge is: {df_nyp['MaximumNegotiatedCharge'].var()}")
print(f"The standard deviation of MaximumNegotiatedCharge is: {df_nyp['MaximumNegotiatedCharge'].std()}")
print(f"The IQR of MaximumNegotiatedCharge is: {df_nyp['MaximumNegotiatedCharge'].quantile(0.75) - df_nyp['MaximumNegotiatedCharge'].quantile(0.25)}")

The mean of MaximumNegotiatedCharge	is : 23902.77424282837
The median of MaximumNegotiatedCharge	is : 4086.7313
The mode of MaximumNegotiatedCharge	is : 11924.0
The range of MaximumNegotiatedCharge is: 1713458.5920000002
The variance of MaximumNegotiatedCharge is: 5092967278.384099
The standard deviation of MaximumNegotiatedCharge is: 71365.02839895812
The IQR of MaximumNegotiatedCharge is: 18618.398999999998


### *df_msmc*

In [23]:
# distribution of RevenueCodeDRGType

df_msmc.groupby(['RevenueCodeDRGType']).describe()

GrossCharge                                                \
                         count         mean           std     min        25%   
RevenueCodeDRGType                                                             
250                        2.0    10.400000      4.412346    7.28     8.8400   
260                       18.0   574.358889    315.333955  253.12   299.2500   
272                        4.0  3498.950000   6584.163770   61.30   101.8900   
278                       16.0  5795.913750  14727.762487   23.40    96.8500   
289                        1.0   624.000000           NaN  624.00   624.0000   
...                        ...          ...           ...     ...        ...   
920                        2.0   640.315000    524.934861  269.13   454.7225   
921                       19.0  1394.644211    422.736078  624.50  1038.3450   
940                        1.0   132.130000           NaN  132.13   132.1300   
942                        1.0   166.400000           NaN  166.40   166.4000   
943                        4.0   281.655000    156.389285   96.60   209.2650   

                                                  AETNABETTERHEALTHMEDICAID  \
                         50%        75%       max                     count   
RevenueCodeDRGType                                                            
250                   10.400    11.9600     13.52                       2.0   
260                  445.000   787.2200   1126.25                      18.0   
272                  281.310  3678.3700  13371.88                       4.0   
278                  161.720   728.5200  43494.10                      16.0   
289                  624.000   624.0000    624.00                       1.0   
...                      ...        ...       ...                       ...   
920                  640.315   825.9075   1011.50                       2.0   
921                 1279.860  1771.5000   1964.16                      19.0   
940                  132.130   132.1300    132.13                       1.0   
942                  166.400   166.4000    166.40                       1.0   
943                  277.770   350.1600    474.48                       4.0   

                                ... MaximumNegotiatedCharge              \
                          mean  ...                     75%         max   
RevenueCodeDRGType              ...                                       
250                   0.728000  ...               10.166000     11.4920   
260                  40.205122  ...              669.137000    957.3125   
272                 244.926500  ...             3126.614500  11366.0980   
278                 405.713963  ...              619.242000  36969.9850   
289                  43.680000  ...              530.400000    530.4000   
...                        ...  ...                     ...         ...   
920                  44.822050  ...              702.021375    859.7750   
921                  97.625095  ...             1505.775000   1669.5360   
940                   9.249100  ...              112.310500    112.3105   
942                  11.648000  ...              141.440000    141.4400   
943                  19.715850  ...              297.636000    403.3080   

                   MinimumNegotiatedCharge                                    \
                                     count        mean          std      min   
RevenueCodeDRGType                                                             
250                                    2.0    0.728000     0.308864   0.5096   
260                                   18.0   40.205122    22.073377  17.7184   
272                                    4.0  244.926500   460.891464   4.2910   
278                                   16.0  405.713963  1030.943374   1.6380   
289                                    1.0   43.680000          NaN  43.6800   
...                                    ...         ...          ...      ...   
920                 

In [24]:
# value_counts of ChargeDescription
cdesc_counts = df_msmc['ChargeDescription'].value_counts()
print(cdesc_counts)

HC PHENOBARBITAL                                                               2
HC MR BRAIN W/WO CONTRAST                                                      2
HC 3D RENDERING W POST PROCESS                                                 2
HC C3 NEPHRITIC FACTOR                                                         2
HC PLACE DEVICE/MARKER, NON PROS                                               2
                                                                              ..
HC ANTIMYELOPEROX AB QT                                                        1
HC ANTI-MITOCHONDRIA AB                                                        1
HC C1Q                                                                         1
HC ANTICARDIOLIPIN AB                                                          1
HC REM MNTR PHYSIOL PARAM 1ST DEV SUPPLY  WITH DAILY RECORDINGS; EA 30 DAYS    1
Name: ChargeDescription, Length: 1995, dtype: int64


In [25]:
# value_counts of BillingCode
bcode_counts = df_msmc['BillingCode'].value_counts()
print(bcode_counts)

G0463    13
86255    13
83520    12
94640    10
78830     9
         ..
88364     1
88365     1
88367     1
88368     1
99453     1
Name: BillingCode, Length: 1391, dtype: int64


In [26]:
# statistics of GrossCharge
print(f"The mean of GrossCharge is : {df_msmc['GrossCharge'].mean()}")
print(f"The median of GrossCharge is : {df_msmc['GrossCharge'].median()}")
print(f"The mode of GrossCharge is : {df_msmc['GrossCharge'].mode().iloc[0]}")
print(f"The range of GrossCharge is: {df_msmc['GrossCharge'].max() - df_msmc['GrossCharge'].min()}")
print(f"The variance of GrossCharge is: {df_msmc['GrossCharge'].var()}")
print(f"The standard deviation of GrossCharge is: {df_msmc['GrossCharge'].std()}")
print(f"The IQR of GrossCharge is: {df_msmc['GrossCharge'].quantile(0.75) - df_msmc['GrossCharge'].quantile(0.25)}")

The mean of GrossCharge is : 1753.8667099350973
The median of GrossCharge is : 253.12
The mode of GrossCharge is : 12.48
The range of GrossCharge is: 62035.14
The variance of GrossCharge is: 29649627.837086324
The standard deviation of GrossCharge is: 5445.147182316225
The IQR of GrossCharge is: 987.505


In [27]:
# statistics of CashPrice
print(f"The mean of CashPrice is : {df_msmc['CashPrice'].mean()}")
print(f"The median of CashPrice is : {df_msmc['CashPrice'].median()}")
print(f"The mode of CashPrice is : {df_msmc['CashPrice'].mode().iloc[0]}")
print(f"The range of CashPrice is: {df_msmc['CashPrice'].max() - df_msmc['CashPrice'].min()}")
print(f"The variance of CashPrice is: {df_msmc['CashPrice'].var()}")
print(f"The standard deviation of CashPrice is: {df_msmc['CashPrice'].std()}")
print(f"The IQR of CashPrice is: {df_msmc['CashPrice'].quantile(0.75) - df_msmc['CashPrice'].quantile(0.25)}")

The mean of CashPrice is : 809.2674413379931
The median of CashPrice is : 109.2
The mode of CashPrice is : 125
The range of CashPrice is: 31017.57
The variance of CashPrice is: 7351213.61745605
The standard deviation of CashPrice is: 2711.312157877814
The IQR of CashPrice is: 422.305


In [28]:
# statistics of MinimumNegotiatedCharge
print(f"The mean of MinimumNegotiatedCharge	is : {df_msmc['MinimumNegotiatedCharge'].mean()}")
print(f"The median of MinimumNegotiatedCharge	is : {df_msmc['MinimumNegotiatedCharge'].median()}")
print(f"The mode of MinimumNegotiatedCharge	is : {df_msmc['MinimumNegotiatedCharge'].mode().iloc[0]}")
print(f"The range of MinimumNegotiatedCharge is: {df_msmc['MinimumNegotiatedCharge'].max() - df_msmc['MinimumNegotiatedCharge'].min()}")
print(f"The variance of MinimumNegotiatedCharge is: {df_msmc['MinimumNegotiatedCharge'].var()}")
print(f"The standard deviation of MinimumNegotiatedCharge is: {df_msmc['MinimumNegotiatedCharge'].std()}")
print(f"The IQR of MinimumNegotiatedCharge is: {df_msmc['MinimumNegotiatedCharge'].quantile(0.75) - df_msmc['MinimumNegotiatedCharge'].quantile(0.25)}")

The mean of MinimumNegotiatedCharge	is : 122.77066969545683
The median of MinimumNegotiatedCharge	is : 17.718400000000003
The mode of MinimumNegotiatedCharge	is : 0.8736000000000002
The range of MinimumNegotiatedCharge is: 4342.4598000000005
The variance of MinimumNegotiatedCharge is: 145283.17640172297
The standard deviation of MinimumNegotiatedCharge is: 381.1603027621357
The IQR of MinimumNegotiatedCharge is: 69.12535000000001


In [29]:
# statistics of MaximumNegotiatedCharge
print(f"The mean of MaximumNegotiatedCharge	is : {df_msmc['MaximumNegotiatedCharge'].mean()}")
print(f"The median of MaximumNegotiatedCharge	is : {df_msmc['MaximumNegotiatedCharge'].median()}")
print(f"The mode of MaximumNegotiatedCharge	is : {df_msmc['MaximumNegotiatedCharge'].mode().iloc[0]}")
print(f"The range of MaximumNegotiatedCharge is: {df_msmc['MaximumNegotiatedCharge'].max() - df_msmc['MaximumNegotiatedCharge'].min()}")
print(f"The variance of MaximumNegotiatedCharge is: {df_msmc['MaximumNegotiatedCharge'].var()}")
print(f"The standard deviation of MaximumNegotiatedCharge is: {df_msmc['MaximumNegotiatedCharge'].std()}")
print(f"The IQR of MaximumNegotiatedCharge is: {df_msmc['MaximumNegotiatedCharge'].quantile(0.75) - df_msmc['MaximumNegotiatedCharge'].quantile(0.25)}")

The mean of MaximumNegotiatedCharge	is : 1490.7867034448327
The median of MaximumNegotiatedCharge	is : 215.152
The mode of MaximumNegotiatedCharge	is : 10.608
The range of MaximumNegotiatedCharge is: 52729.869
The variance of MaximumNegotiatedCharge is: 21421856.112294868
The standard deviation of MaximumNegotiatedCharge is: 4628.375104968791
The IQR of MaximumNegotiatedCharge is: 839.37925


### **Conclusions:**

# **2. SQLite Database Operations**

## 2.1 Local SQLite database

In [33]:
# creating a local database
conn = sqlite3.connect('health.db')
c = conn.cursor()

## 2.2 Manual table creation

In [32]:
# creating table

c.execute("""
            CREATE TABLE icecream
              (
                icecream_name text,
                icecream_type text,
                flavor text,
                servings_per_box real,
                cost_per_box real
              );
        """)

conn.commit()

In [34]:
# inserting data into icecream

sql_query = """

INSERT INTO icecream (
  'icecream_name',
  'icecream_type',
  'flavor',
  'servings_per_box',
  'cost_per_box'
  )
  values (
    'Dazs-genhaa',
    'bars',
    'vanilla',
    8,
    10
  );


"""
print(sql_query)



INSERT INTO icecream (
  'icecream_name',
  'icecream_type',
  'flavor',
  'servings_per_box',
  'cost_per_box'
  )
  values (
    'Dazs-genhaa',
    'bars',
    'vanilla',
    8,
    10
  );





In [35]:
# executing sql_query
c.execute(sql_query)
conn.commit()

In [39]:
# inserting more information
sql_query2 = """

INSERT INTO icecream (
  'icecream_name',
  'icecream_type',
  'flavor',
  'servings_per_box',
  'cost_per_box'
  )
  values (
    'Jen & Berrys',
    'sandwich',
    'coffee',
    12,
    18
  );


"""
print(sql_query2)



INSERT INTO icecream (
  'icecream_name',
  'icecream_type',
  'flavor',
  'servings_per_box',
  'cost_per_box'
  )
  values (
    'Jen & Berrys',
    'sandwich',
    'coffee',
    12,
    18
  );





In [40]:
# executing sql_query2

c.execute(sql_query2)
conn.commit()

In [43]:
# checking if rows have been populated
query = """

  SELECT *
  from icecream;

  """

c.execute(query)
print(c.fetchall())

[('Dazs-genhaa', 'bars', 'vanilla', 8.0, 10.0), ('Jen & Berrys', 'sandwich', 'coffee', 12.0, 18.0)]


In [44]:
# creating engine to connect to db

engine = create_engine('sqlite:///health.db')

In [45]:
# displaying table
icecream = pd.read_sql(query, conn)
icecream

,icecream_name,icecream_type,flavor,servings_per_box,cost_per_box
0,Dazs-genhaa,bars,vanilla,8.0,10.0
1,Jen & Berrys,sandwich,coffee,12.0,18.0


## 2.3 Automatic table creation

In [46]:
#creating new column
columnNames = list(df_msmc)
idVars = columnNames[:5]
valueVars = columnNames[5:]

icecream_modified = df_msmc.melt(id_vars=idVars, value_vars=valueVars)

icecream_modified.columns

icecream_modified.rename(columns={'icecream_name': 'ChargeDescription',
                                  'icecream_type': 'LongDescription',
                                  'flavor':'BillingCode',
                                  'servings_per_box':'RevenueCodeDRGType',
                                  'cost_per_box': 'GrossCharge'}, inplace=True)

icecream_modified

,ChargeDescription,LongDescription,BillingCode,RevenueCodeDRGType,GrossCharge,variable,value
0,HC BRIDGE ASSESSMENT,Psychiatric diagnostic evaluation\n,90791,513,156.00,CashPrice,175
1,HC PSYCHIATRIC DIAGNOSTIC EVALUATION,Psychiatric diagnostic evaluation\n,90791,900,327.60,CashPrice,315
2,HC PSYCHIATRIC DIAGNOSTIC EVALUATION W/ MEDICA...,Psychiatric diagnostic evaluation with medical...,90792,900,327.60,CashPrice,315
3,HC ELECTROCONVULSIVE THERAPY,Electroconvulsive therapy (includes necessary ...,90870,901,1154.40,CashPrice,577.2
4,HC PSYCH TREATMENT & EDUCATION,Training and educational services related to t...,G0177,942,166.40,CashPrice,83.2
...,...,...,...,...,...,...,...
102148,HC PHLEBOTOMY,"Phlebotomy, therapeutic (separate procedure)",99195,940,132.13,MinimumNegotiatedCharge,9.2491
102149,HC CARDIAC REHAB PHASE I,Physician or other qualified health care profe...,93797,943,246.82,MinimumNegotiatedCharge,17.2774
102150,HC CARDIAC REHAB PHASE 2,Physician or other qualified health care profe...,93798,943,474.48,MinimumNegotiatedCharge,33.2136
102151,HC REM MNTR PHYSIOL PARAM INITIAL SETUP AND ED...,Remote monitoring of physiologic parameter(s) ...,99453,943,308.72,MinimumNegotiatedCharge,21.6104


In [48]:
# replacing icecream with icecream_modified
icecream_modified.to_sql('icecream', conn, if_exists='replace')

102153

In [50]:
#testing modified

modified_query = """
  select *
  from icecream
  where RevenueCodeDRGType = '900'
  limit 10;
"""
check = pd.read_sql(modified_query, conn)
check

,index,ChargeDescription,LongDescription,BillingCode,RevenueCodeDRGType,GrossCharge,variable,value
0,1,HC PSYCHIATRIC DIAGNOSTIC EVALUATION,Psychiatric diagnostic evaluation\n,90791,900,327.6,CashPrice,315
1,2,HC PSYCHIATRIC DIAGNOSTIC EVALUATION W/ MEDICA...,Psychiatric diagnostic evaluation with medical...,90792,900,327.6,CashPrice,315
2,2004,HC PSYCHIATRIC DIAGNOSTIC EVALUATION,Psychiatric diagnostic evaluation\n,90791,900,327.6,AETNABETTERHEALTHMEDICAID,22.932
3,2005,HC PSYCHIATRIC DIAGNOSTIC EVALUATION W/ MEDICA...,Psychiatric diagnostic evaluation with medical...,90792,900,327.6,AETNABETTERHEALTHMEDICAID,22.932
4,4007,HC PSYCHIATRIC DIAGNOSTIC EVALUATION,Psychiatric diagnostic evaluation\n,90791,900,327.6,AETNACOMMERCIAL,98.28
5,4008,HC PSYCHIATRIC DIAGNOSTIC EVALUATION W/ MEDICA...,Psychiatric diagnostic evaluation with medical...,90792,900,327.6,AETNACOMMERCIAL,98.28
6,6010,HC PSYCHIATRIC DIAGNOSTIC EVALUATION,Psychiatric diagnostic evaluation\n,90791,900,327.6,AETNAINTERNATIONAL,278.46
7,6011,HC PSYCHIATRIC DIAGNOSTIC EVALUATION W/ MEDICA...,Psychiatric diagnostic evaluation with medical...,90792,900,327.6,AETNAINTERNATIONAL,278.46
8,8013,HC PSYCHIATRIC DIAGNOSTIC EVALUATION,Psychiatric diagnostic evaluation\n,90791,900,327.6,AETNAMEDICARE,45.864
9,8014,HC PSYCHIATRIC DIAGNOSTIC EVALUATION W/ MEDICA...,Psychiatric diagnostic evaluation with medical...,90792,900,327.6,AETNAMEDICARE,45.864
